In [1]:
import clickhouse_connect
import os
import keyring

host = keyring.get_password("clickhouse-confirm-host", os.environ["USER"])
password = keyring.get_password("clickhouse-confirm-password", os.environ["USER"])
client = clickhouse_connect.get_client(
    host=host,
    port=8443,
    username="default",
    password=password,
)
# client = clickhouse_connect.get_client(host='127.0.0.1', port=8123)

In [2]:
client.command("CREATE TABLE test_table (key UInt16, value String) ENGINE Memory")
data = [[100, "value1"], [200, "value2"]]
client.insert("test_table", data)
print(client.query("SELECT * FROM test_table").result_set)

[(100, 'value1'), (200, 'value2')]


In [2]:
import pandas as pd

df = pd.read_parquet("./dbtest.parquet")

In [2]:
df[:2000000].to_parquet("./dbtestsmall.parquet")

In [7]:
query = (
    "create table tiles ("
    + ",".join([f"{c} Float32" for c in df.columns])
    + ") engine = MergeTree() order by orig_lam"
)
query

'create table tiles (alpha0 Float32,orig_lam Float32,B_lam_0 Float32,B_lam_1 Float32,B_lam_2 Float32,B_lam_3 Float32,B_lam_4 Float32,B_lam_5 Float32,B_lam_6 Float32,B_lam_7 Float32,B_lam_8 Float32,B_lam_9 Float32,B_lam_10 Float32,B_lam_11 Float32,B_lam_12 Float32,B_lam_13 Float32,B_lam_14 Float32,B_lam_15 Float32,B_lam_16 Float32,B_lam_17 Float32,B_lam_18 Float32,B_lam_19 Float32,B_lam_20 Float32,B_lam_21 Float32,B_lam_22 Float32,B_lam_23 Float32,B_lam_24 Float32,B_lam_25 Float32,B_lam_26 Float32,B_lam_27 Float32,B_lam_28 Float32,B_lam_29 Float32,B_lam_30 Float32,B_lam_31 Float32,B_lam_32 Float32,B_lam_33 Float32,B_lam_34 Float32,B_lam_35 Float32,B_lam_36 Float32,B_lam_37 Float32,B_lam_38 Float32,B_lam_39 Float32,B_lam_40 Float32,B_lam_41 Float32,B_lam_42 Float32,B_lam_43 Float32,B_lam_44 Float32,B_lam_45 Float32,B_lam_46 Float32,B_lam_47 Float32,B_lam_48 Float32,B_lam_49 Float32,twb_min_lam Float32,twb_mean_lam Float32,twb_max_lam Float32,sim_sizes Float32,todo Float32,grid_pt_idx Flo

In [8]:
client.command(query)

''

In [3]:
%%time
client.insert_df("tiles", df[:100000])

CPU times: user 1.85 s, sys: 40.3 ms, total: 1.89 s
Wall time: 3.21 s


In [4]:
import pyarrow

In [6]:
%%time
client.insert_arrow("tiles", pyarrow.Table.from_pandas(df[:100000]))

CPU times: user 53.7 ms, sys: 43.9 ms, total: 97.7 ms
Wall time: 2.5 s


In [2]:
%%time
sel_df = client.query_df("select * from tiles order by orig_lam limit 10000")
sel_df.shape

CPU times: user 63.7 ms, sys: 10.2 ms, total: 73.9 ms
Wall time: 867 ms


(10000, 69)

In [3]:
%%time
client.query(
    "select " + ",".join([f"min(B_lam_{i})" for i in range(50)]) + " from tiles"
).result_set

CPU times: user 2.24 ms, sys: 984 µs, total: 3.23 ms
Wall time: 3.53 s


[(0.06231215223670006,
  0.055375631898641586,
  0.0625298023223877,
  0.061695247888565063,
  0.0625298023223877,
  0.060130525380373,
  0.05916070193052292,
  0.056377604603767395,
  0.05082782730460167,
  0.05162762105464935,
  0.05911017581820488,
  0.06067570298910141,
  0.06466516107320786,
  0.05746819078922272,
  0.06012817844748497,
  0.051577892154455185,
  0.05938999727368355,
  0.06441992521286011,
  0.05864203721284866,
  0.05549140274524689,
  0.06099971383810043,
  0.05746819078922272,
  0.05916070193052292,
  0.0575517937541008,
  0.062339331954717636,
  0.05890149250626564,
  0.060382794588804245,
  0.061473529785871506,
  0.06222115457057953,
  0.06067570298910141,
  0.05864203721284866,
  0.06321893632411957,
  0.06258217245340347,
  0.06157900020480156,
  0.05329404026269913,
  0.0628439337015152,
  0.061255715787410736,
  0.06314831227064133,
  0.06263559311628342,
  0.0528295561671257,
  0.06314831227064133,
  0.05884512886404991,
  0.05989359691739082,
  0.058642

In [4]:
%%time
client.query_df("select count(*) from tiles")

CPU times: user 1.78 ms, sys: 948 µs, total: 2.73 ms
Wall time: 22.1 ms


,count()
0,19431917


In [5]:
import time

start = time.time()
i = 0
while time.time() - start < 60 * 60:
    frac = 10
    q1 = client.query_df(
        f"select count(*) from tiles where sqrt (power(theta_0+0.5,2)+power(theta_1+0.5,2)+power(theta_2+0.5,2)+power(theta_3+0.5,2)) < {frac}"
    )
    q2 = client.query(
        "select " + ",".join([f"min(B_lam_{i})" for i in range(50)]) + " from tiles"
    ).result_set
    if i % 60 == 0:
        print(i, q1, q2)
    i += 1

0     count()
0  19431917 [(0.06231215223670006, 0.055375631898641586, 0.0625298023223877, 0.061695247888565063, 0.0625298023223877, 0.060130525380373, 0.05916070193052292, 0.056377604603767395, 0.05082782730460167, 0.05162762105464935, 0.05911017581820488, 0.06067570298910141, 0.06466516107320786, 0.05746819078922272, 0.06012817844748497, 0.051577892154455185, 0.05938999727368355, 0.06441992521286011, 0.05864203721284866, 0.05549140274524689, 0.06099971383810043, 0.05746819078922272, 0.05916070193052292, 0.0575517937541008, 0.062339331954717636, 0.05890149250626564, 0.060382794588804245, 0.061473529785871506, 0.06222115457057953, 0.06067570298910141, 0.05864203721284866, 0.06321893632411957, 0.06258217245340347, 0.06157900020480156, 0.05329404026269913, 0.0628439337015152, 0.061255715787410736, 0.06314831227064133, 0.06263559311628342, 0.0528295561671257, 0.06314831227064133, 0.05884512886404991, 0.05989359691739082, 0.05864203721284866, 0.060318052768707275, 0.06224958598613739, 0.06

In [9]:
# 393.75 at 6:24, wait a bit
(495 - 393.75) * 0.00239

0.24198750000000002

In [6]:
usage = 1680 - 828.75
price = 0.00239
time = 60
estimated_cu_per_min = usage / time

2.0344875000000004

## Materialized View

In [45]:
import clickhouse_connect

client = clickhouse_connect.get_client(
    host=host, port=8443, username="default", password=password
)

In [23]:
client.command("drop view Blamss")

''

In [64]:
query = (
    "create materialized view if not exists Blamss order by (Blamss0) POPULATE as select "
    + ",".join([f"min(B_lam_{i}) as Blamss{i}" for i in range(50)])
    + " from tiles"
)

In [65]:
client.command(query)

''

In [91]:
%%time
client.query(
    "select " + ",".join([f"min(Blamss{i})" for i in range(50)]) + " from Blamss"
).result_set

CPU times: user 2.37 ms, sys: 1.49 ms, total: 3.86 ms
Wall time: 265 ms


[(0.06231215223670006,
  0.055375631898641586,
  0.0625298023223877,
  0.061695247888565063,
  0.0625298023223877,
  0.060130525380373,
  0.05916070193052292,
  0.056377604603767395,
  0.05082782730460167,
  0.05162762105464935,
  0.05911017581820488,
  0.06067570298910141,
  0.06466516107320786,
  0.05746819078922272,
  0.06012817844748497,
  0.051577892154455185,
  0.05938999727368355,
  0.06441992521286011,
  0.05864203721284866,
  0.05549140274524689,
  0.06099971383810043,
  0.05746819078922272,
  0.05916070193052292,
  0.0575517937541008,
  0.062339331954717636,
  0.05890149250626564,
  0.060382794588804245,
  0.061473529785871506,
  0.06222115457057953,
  0.06067570298910141,
  0.05864203721284866,
  0.06321893632411957,
  0.06258217245340347,
  0.06157900020480156,
  0.05329404026269913,
  0.0628439337015152,
  0.061255715787410736,
  0.06314831227064133,
  0.06263559311628342,
  0.0528295561671257,
  0.06314831227064133,
  0.05884512886404991,
  0.05989359691739082,
  0.058642

In [90]:
%%time
client.insert_df("tiles", df[:100000])

CPU times: user 1.82 s, sys: 45.5 ms, total: 1.86 s
Wall time: 4.48 s


## Add index

In [2]:
import clickhouse_connect

client = clickhouse_connect.get_client(
    host=host,
    port=8443,
    username="default",
    password=password,
)

In [3]:
%%time
client.query("select min(B_lam_0) from tiles")

CPU times: user 1.95 ms, sys: 1.1 ms, total: 3.05 ms
Wall time: 64.2 ms


In [4]:
for i in range(50):
    client.query(f"alter table tiles drop index Blam{i}idx")

Code: 517. DB::Exception: Metadata on replica is not up to date with common metadata in Zookeeper. It means that this replica still not applied some of previous alters. Probably too many alters executing concurrently (highly not recommended). You can retry this error. (CANNOT_ASSIGN_ALTER) (version 22.11.1.19266 (official build))



DatabaseError: :HTTPDriver for https://gznwzfxi1x.us-east-1.aws.clickhouse.cloud:8443 returned response code 500)
 Code: 517. DB::Exception: Metadata on replica is not up to date with common metadata in Zookeeper. It means that this replica still not applied some of previous alters. Probably too many alters executing concurrently (highly not recommended

In [5]:
for i in range(1):
    client.query(
        f"alter table tiles add index Blam{i}idx B_lam_{i} TYPE minmax GRANULARITY 1"
    )
for i in range(1):
    client.query(f"alter table tiles materialize index Blam{i}idx")

In [10]:
%%time
client.query("select min(B_lam_0) from tiles")

CPU times: user 1.6 ms, sys: 1.06 ms, total: 2.66 ms
Wall time: 39 ms


In [20]:
%%time
bl0 = client.query("select B_lam_0 from tiles limit 1").result_set[0][0]

CPU times: user 19.1 ms, sys: 1.8 ms, total: 20.9 ms
Wall time: 82.8 ms


In [21]:
bl0

0.06693554669618607

In [40]:
%%time
client.query(
    f"select min(B_lam_0) from tiles where B_lam_0 < 0.063",
    settings={"force_data_skipping_indices": "Blam0idx"},
).result_set

CPU times: user 1.86 ms, sys: 1.05 ms, total: 2.91 ms
Wall time: 37.6 ms


[(0.06231215223670006,)]

In [11]:
%%time
client.query(
    "select min(B_lam_0) from tiles",
    settings={"force_data_skipping_indices": "Blam0idx"},
).result_set

Code: 277. DB::Exception: Index `Blam0idx` is not used and setting 'force_data_skipping_indices' contains it. (INDEX_NOT_USED) (version 22.11.1.19266 (official build))



DatabaseError: :HTTPDriver for https://gznwzfxi1x.us-east-1.aws.clickhouse.cloud:8443 returned response code 500)
 Code: 277. DB::Exception: Index `Blam0idx` is not used and setting 'force_data_skipping_indices' contains it. (INDEX_NOT_USED) (version 22.11.1.19266 (official build))


In [33]:
%%time
client.query(
    "select " + ",".join([f"min(B_lam_{i})" for i in range(50)]) + " from tiles"
).result_set

CPU times: user 3.9 ms, sys: 1.52 ms, total: 5.42 ms
Wall time: 1.2 s


[(0.06231215223670006,
  0.055375631898641586,
  0.0625298023223877,
  0.061695247888565063,
  0.0625298023223877,
  0.060130525380373,
  0.05916070193052292,
  0.056377604603767395,
  0.05082782730460167,
  0.05162762105464935,
  0.05911017581820488,
  0.06067570298910141,
  0.06466516107320786,
  0.05746819078922272,
  0.06012817844748497,
  0.051577892154455185,
  0.05938999727368355,
  0.06441992521286011,
  0.05864203721284866,
  0.05549140274524689,
  0.06099971383810043,
  0.05746819078922272,
  0.05916070193052292,
  0.0575517937541008,
  0.062339331954717636,
  0.05890149250626564,
  0.060382794588804245,
  0.061473529785871506,
  0.06222115457057953,
  0.06067570298910141,
  0.05864203721284866,
  0.06321893632411957,
  0.06258217245340347,
  0.06157900020480156,
  0.05329404026269913,
  0.0628439337015152,
  0.061255715787410736,
  0.06314831227064133,
  0.06263559311628342,
  0.0528295561671257,
  0.06314831227064133,
  0.05884512886404991,
  0.05989359691739082,
  0.058642

## Test locking mechanism

janky locking mechanisms from https://blog.qryn.dev/shared-resource-lock-powered-by-clickhouse

the comments suggest `query_id`. this doesn't work for me.

In [ ]:
clickhouse_connect

In [97]:
client.command("drop table _mtx")

Code: 60. DB::Exception: Table default._mtx doesn't exist. (UNKNOWN_TABLE) (version 22.11.1.19266 (official build))



DatabaseError: :HTTPDriver for https://gznwzfxi1x.us-east-1.aws.clickhouse.cloud:8443 returned response code 404)
 Code: 60. DB::Exception: Table default._mtx doesn't exist. (UNKNOWN_TABLE) (version 22.11.1.19266 (official build))


In [1]:
import clickhouse_connect

client = clickhouse_connect.get_client(
    host=host, port=8443, username="default", password=password
)

In [22]:
import time
import clickhouse_connect
from contextlib import contextmanager
from multiprocessing.pool import ThreadPool

query = "select " + ",".join([f"min(Blamss{i})" for i in range(50)]) + " from Blamss"

client.command("drop table if exists _mtx0", settings={"wait_end_of_query": 1})


@contextmanager
def claim_ch_mutex(client, mutex_id, timeout=60, claim_interval=60, retry_interval=0.5):
    start = time.time()
    while time.time() - start < timeout:
        try:
            client.command(
                f"CREATE TABLE _mtx{mutex_id} (creation_time int, claim_interval int) order by creation_time"
            )
            client.insert(f"_mtx{mutex_id}", [[int(time.time()), claim_interval]])
            print("created and inserted")
            break
        except clickhouse_connect.driver.exceptions.DatabaseError as e:
            if f"_mtx{mutex_id} already exists" not in str(e):
                raise
            try:
                mtx_df = client.query_df(f"select * from _mtx{mutex_id}")
            except clickhouse_connect.driver.exceptions.DatabaseError as e:
                if f"_mtx{mutex_id} doesn't exist" in str(e):
                    print("table doesn't exist now! retrying")
                    continue
                raise
            if mtx_df.shape[0] > 0:
                mtx = mtx_df.iloc[0]
                if time.time() - mtx["creation_time"] > mtx["claim_interval"]:
                    print(f"Mutex{mutex_id} claim expired, deleting claim.")
                    client.command("DROP TABLE _mtx")
            print(
                f"Mutex{mutex_id} is locked. Sleeping for {retry_interval}s and then retrying."
            )
            time.sleep(retry_interval)
    if time.time() - start > timeout:
        raise Exception(f"Could not claim mutex{mutex_id} within {timeout}s")
    try:
        yield
    finally:
        client.command(f"DROP TABLE _mtx{mutex_id}")


def connect_and_query(worker_id):
    client = clickhouse_connect.get_client(
        host=host, port=8443, username="default", password=password
    )
    with claim_ch_mutex(client, 0):
        return client.query_df(query)


N = 4
ThreadPool(N).map(lambda x: connect_and_query(x), range(N))

Code: 57. DB::Exception: Table default._mtx0 already exists. (TABLE_ALREADY_EXISTS) (version 22.11.1.19266 (official build))

Code: 57. DB::Exception: Table default._mtx0 already exists. (TABLE_ALREADY_EXISTS) (version 22.11.1.19266 (official build))

Code: 57. DB::Exception: Table default._mtx0 already exists. (TABLE_ALREADY_EXISTS) (version 22.11.1.19266 (official build))



Mutex0 is locked. Sleeping for 0.5s and then retrying.
Mutex0 is locked. Sleeping for 0.5s and then retrying.
Mutex0 is locked. Sleeping for 0.5s and then retrying.
created and inserted


Code: 57. DB::Exception: Table default._mtx0 already exists. (TABLE_ALREADY_EXISTS) (version 22.11.1.19266 (official build))



Mutex0 is locked. Sleeping for 0.5s and then retrying.


Code: 57. DB::Exception: Table default._mtx0 already exists. (TABLE_ALREADY_EXISTS) (version 22.11.1.19266 (official build))



...

Code: 57. DB::Exception: Table default._mtx0 already exists. (TABLE_ALREADY_EXISTS) (version 22.11.1.19266 (official build))



Mutex0 is locked. Sleeping for 0.5s and then retrying.


Code: 57. DB::Exception: Table default._mtx0 already exists. (TABLE_ALREADY_EXISTS) (version 22.11.1.19266 (official build))



Mutex0 is locked. Sleeping for 0.5s and then retrying.
created and inserted
created and inserted


[   min(Blamss0)  min(Blamss1)  min(Blamss2)  min(Blamss3)  min(Blamss4)  \
 0      0.062312      0.055376       0.06253      0.061695       0.06253   
 
    min(Blamss5)  min(Blamss6)  min(Blamss7)  min(Blamss8)  min(Blamss9)  ...  \
 0      0.060131      0.059161      0.056378      0.050828      0.051628  ...   
 
    min(Blamss40)  min(Blamss41)  min(Blamss42)  min(Blamss43)  min(Blamss44)  \
 0       0.063148       0.058845       0.059894       0.058642       0.060318   
 
    min(Blamss45)  min(Blamss46)  min(Blamss47)  min(Blamss48)  min(Blamss49)  
 0        0.06225       0.062994       0.058642       0.057468       0.055376  
 
 [1 rows x 50 columns],
    min(Blamss0)  min(Blamss1)  min(Blamss2)  min(Blamss3)  min(Blamss4)  \
 0      0.062312      0.055376       0.06253      0.061695       0.06253   
 
    min(Blamss5)  min(Blamss6)  min(Blamss7)  min(Blamss8)  min(Blamss9)  ...  \
 0      0.060131      0.059161      0.056378      0.050828      0.051628  ...   
 
    min(Blamss